In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense

# pretrained InceptionV3 사용
base_model = InceptionV3(weights='imagenet',
                         include_top=False,
                         input_shape=(299, 299, 3))
# 313 layer 중 256개 까지만 base model로 사용
base_model = base_model.layers[:256]

new_output = GlobalAveragePooling2D()(base_model[-1].output)
new_output = Dense(7, activation='softmax')(new_output) # 무증상 + 유증상(6개 Class)
model = tf.keras.models.Model(base_model.inputs, new_output)

# 213 layer 이전은 학습 동결
for idx, layer in enumerate(model.layers):
    if isinstance(layer, tf.keras.layers.BatchNormalization): # BatchNormalization layer는 학습
        layer.trainable = True
    else:
        if idx < 213 :
            layer.trainable = False
        else :
            layer.trainable = True

model.compile(loss='categorical_crossentropy', metrics=['accuracy'],
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.001))

# Learnling Rate Reduce 적용
lr_reduction = ReduceLROnPlateau(monitor='val_loss',
                            patience=5,
                            verbose=1,
                            factor=0.5,
                            min_lr=0.000001)
# EarlyStop 적용
es = EarlyStopping(monitor='val_accuracy',
                    min_delta=0,
                    patience=8,
                    verbose=1,
                    restore_best_weights=True)

checkpoint = ModelCheckpoint("/content/drive/MyDrive/model_checkpoint.h5",
                             save_weights_only=False,
                             save_best_only=True,
                             monitor="val_accuracy",
                             verbose=1)

batch_size = 128

data_path = "/content/drive/MyDrive/big_project/dataset"
# Data Augmentation 진행
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
    rotation_range=20,  
    width_shift_range=0.2,  
    height_shift_range=0.2, 
    shear_range=0.2,  
    zoom_range=0.2, 
    horizontal_flip=True,  
    vertical_flip=True
)

train_generator = datagen.flow_from_directory(
    data_path,
    target_size=(299, 299),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

valid_generator = datagen.flow_from_directory(
    data_path,
    target_size=(299, 299),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
    )


epochs = 100
history= model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=valid_generator,
    validation_steps=valid_generator.samples // batch_size,
    callbacks=[lr_reduction, es],
    epochs=epochs,
).history

# 모델 저장
model.save("/content/drive/MyDrive/model_inception_layer213_256.h5")